# Create Individual Tools

For each site, create:
- read only view of the data and map
- editable view of the data and map(with updated template for unique numbers)
- dashboard of the sites stats

#### Import statments

In [1]:
import arcgis
gis = arcgis.gis.GIS(profile="ag")

# import modules
import json
import copy
import os
from getpass import getpass

from arcgis.geometry import Geometry
from arcgis.features import FeatureLayerCollection

#### Set paths and load in Data Model

In [1]:
basePath = r'C:\Users\rjc.EAGLE\Documents\GitHub\AgricultureDataModel'

data = {}
data.update(json.load(open('{0}\\json\\{1}.json'.format(basePath, 'datamodel'))))
data.update(json.load(open('{0}\\json\\{1}.json'.format(basePath, 'tables'))))
data.update(json.load(open('{0}\\json\\{1}.json'.format(basePath, 'domains'))))
data

NameError: name 'json' is not defined

In [3]:
#administrativeInformationItemID = data['services'][0]['itemID']
administrativeInformationItemID = 'fa6d74cff8724478877b2dfd51c623a5'
administrativeInformation = gis.content.get(administrativeInformationItemID)
sites = administrativeInformation.layers[2].query().features

In [6]:
# define the versions that will be created
versions = [
    {'name':'Stakeholder', 'title':'stakeholder', 'capabilities': 'Query,Sync'},            
    {'name':'Fieldworker', 'title':'fieldworker', 'capabilities': 'Create,Delete,Query,Update,Sync,ChangeTracking'}
]

for service in data['services']:
    
    item = gis.content.get(service['id'])
    featureLayerCollection = FeatureLayerCollection.fromitem(item)
    
    for site in sites:
        
        uniqueNumber = site.attributes['unique_number']
        siteQuery = administrativeInformation.layers[2].query("unique_number = {0}".format(uniqueNumber), out_sr=4326).features[0]
        extent = Geometry(siteQuery.geometry).extent
        
        for version in versions:
            
            viewTitle = 'site_{0}_{1}_view'.format(uniqueNumber, version['title'])
            mapTitle = 'site_{0}_{1}_map'.format(uniqueNumber, version['title'])
            
            viewName = 'Site {0} - {1} View'.format(uniqueNumber, version['name'])
            mapName = 'Site {0} - {1} Map'.format(uniqueNumber, version['name']) 
                    
            groupName = "{0} ({1}) - {2}".format(site.attributes['name'], uniqueNumber, version['name'])
            groupDescription = "A group for sharing content relating to site {0}".format(uniqueNumber)
            group = gis.groups.create(groupName, uniqueNumber, groupDescription)
            
            viewItemProperties = {
                'snippet': '{0} view for site {1} ({2})'.format(version['name'], uniqueNumber, site.attributes['name']),
                'tags': '{0},{1}'.format(uniqueNumber, version['title']), 
                'title': '{0}'.format(viewTitle)
            }
            
            view = featureLayerCollection.manager.create_view(viewTitle, extent=extent, capabilities=version['capabilities'])
            
            #view.move(folder='Individual Farm Views')
            view.share(groups=[group])            
            view.update(viewItemProperties) # thumbnail='/arcgis/home/FarmDataMaster.png'
            
            for layer in view.layers:
                
                layerProperties = layer.properties
                
                viewDefinitionQuery = {"viewDefinitionQuery": "unique_number = {0}".format(uniqueNumber)}                
                layer.manager.update_definition(viewDefinitionQuery)
                
                if version['title'] == 'fieldworker':
                    layerTemplates = layerProperties['templates']
                    for template in layerTemplates:
                        template['prototype']['attributes']['unique_number'] = uniqueNumber
                        
                    layerProperties['editingInfo']['lastEditDate'] = ''
                    layer.manager.update_definition(layerProperties)
                
            
        
        
    
    



Exception: Your request has timed out.
(Error Code: 504)

In [10]:
featureLayerCollection.manager.create_view(viewTitle)

Exception: Your request has timed out.
(Error Code: 504)

In [7]:
version['capabilities']

'Query,Sync'

In [8]:
version

{'name': 'Stakeholder', 'title': 'stakeholder', 'capabilities': 'Query,Sync'}